In [2]:
import glob
import pickle
import pandas as pd
import numpy as np
import scipy
import nltk
import re
import random

from typing import List, Tuple
from collections import OrderedDict

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [17]:
df = pd.read_pickle('./data/my_digital_hand/mdh_logs.p')
df.head()

,time_raised_hand,time_interaction_began,time_interaction_ended,im_working_on,my_problem_is,ive_tried,teacher_suggestions
0,2019-09-20 20:54:17,2019-09-20 21:26:59,2019-09-20 21:28:53,A3 Novice and Jedi,Not passing jUnits test - things that should w...,everything,NaN
1,2019-09-23 14:13:16,2019-09-23 14:22:47,2019-09-23 14:32:18,a4 novice,How to start,reading read me,NaN
2,2019-09-23 17:35:49,2019-09-23 17:46:27,2019-09-23 18:04:46,A3 Novice Tests,differentiating between the interface name and...,"Renaming the classes, but that creates more er...",NaN
3,2019-09-23 17:35:50,2019-09-23 17:36:31,2019-09-23 18:14:30,A3 Novice Tests,Having trouble with default methods & Junit,Output good,Go see getWaitTime
4,2019-09-23 19:17:07,2019-09-23 19:18:20,2019-09-23 20:19:05,"A3 Adept, Novice",Creating a constructor for ShortestWaitDispatcher,creating one myself,NaN


In [37]:
# Auxiliary LDA Functions (c&p from the data_analysis file, thanks alex)
def print_topics(model, vectorizer, top_n: int=10)-> List: 
    
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        
    return [vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n-1:-1]]
        
def lda_operation(data_samples, num_features: int=100, num_topics: int=3)-> Tuple: 
    
    tf_vectorizer = CountVectorizer(max_df=.50, min_df=.0, max_features=num_features, stop_words='english', token_pattern=u'(?ui)\\b\\w\w*[a-z]+\\w*\\b')
    
    tf_data_samples = tf_vectorizer.fit_transform(data_samples) 
    tf_feature_names = tf_vectorizer.get_feature_names()

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=10, learning_method='online', learning_offset=10.,random_state=1).fit(tf_data_samples)
    lda.score(tf_data_samples)

    return lda, tf_vectorizer

def save_topics(model, vectorizer, top_n: int=10)-> List:

    words_per_topic = []
    for idx, topic in enumerate(model.components_):
        words = [vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n-1:-1]]
        words_per_topic.append(words)
        
    return words_per_topic

In [38]:
lda_output = []
data_list = df['my_problem_is'].tolist()
lda_output.append(lda_operation(data_list))
    
# Save all of the words found by the topic model for each class
for output in lda_output:
    print_topics(*output)

Topic 0:
[('code', 67.84499957526695), ('output', 48.784141632696006), ('questions', 44.822284003521155), ('array', 41.99147167387723), ('help', 36.87840105609537), ('need', 34.804845995362925), ('sure', 34.10421034421704), ('understand', 33.54866581938843), ('win', 32.81901241819195), ('spot', 31.159635087920602)]
Topic 1:
[('don', 86.1790858765925), ('method', 76.64322522353764), ('know', 74.82295327793463), ('test', 74.23208421833621), ('work', 59.97492076923204), ('gradescope', 38.19694906956188), ('start', 37.925554188997644), ('jedi', 36.45104707169026), ('won', 33.880539586061325), ('pass', 32.961003350827355)]
Topic 2:
[('tests', 165.85982373015204), ('failing', 93.61957394548692), ('junit', 67.88146710526038), ('understanding', 64.12770316345467), ('methods', 62.960723297177985), ('paint', 47.315580643559684), ('adept', 46.72947138889848), ('having', 39.08018184982573), ('novice', 38.112137180156644), ('working', 37.04161641569801)]
